In [4]:
import numpy as np
import pandas as pd
import csv
from scipy import stats
import os
import datetime
import statsmodels.api as sm
import requests
import sys, json
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import random

In [5]:
# IN CASE NEED TO UPDATE CHROMEDRIVER, RUN BELOW 

# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

In [16]:
# DATA PREP FOR DISTILLER SCRAPE 

ls_whisk_name = []
ls_whisk_type = []
ls_whisk_loc = []
ls_comm_rating = []
ls_distiller_score = []
ls_whisk_url = []

# https://distiller.com/search?country_id=1&page=10&sort=total_num_of_ratings&spirit_style_id=1 # THIS IS SINGLE MALT
# spirit style --> 56-63 --> the rums

### Loop through EVERY rum type based on History

We can do this in one ls_url list b/c ls_url can be extended

Note, for rum, we don't do any filtering. Just pick up all rums by first clicking on the rum button when visiting distiller.com home page and then going to the search page

In [17]:
### Start with empty list
ls_url = []

In [18]:
### All Rums in One Go!

ls_sites = list(range(1, 155)) # FIGURE THIS OUT MANUALLY FOR EACH PULL WE DO
ls_url_iter = [f'https://distiller.com/search?page={str(i)}&sort=total_num_of_ratings' for i in ls_sites]
ls_url.extend(ls_url_iter)

In [6]:
# spirit style --> 56-63 --> the rums ### NEED TO GO TO DIFFERENT PAGE

In [19]:
# Print the first page for recon
print(len(ls_url))
ls_url[153]
# ls_url[170]

154


'https://distiller.com/search?page=154&sort=total_num_of_ratings'

In [37]:
### THE BELOW SCRAPER GETS US TO RUM

driver = webdriver.Chrome(r'C:\Users\aclou\AppData\Local\Programs\Python\Python39\Scripts\chromedriver')
driver.maximize_window()
driver.get('https://distiller.com')
time.sleep(8)
xpath_rum = "//div[@class='center-column']/ul[@class='spirit-family-select__options']/li[@class='spirit-family-select__option sugarcane']/button" # this will naturally filter out NON-whiskies
elem = driver.find_element_by_xpath(xpath_rum)
elem.click() ### this gets us to the RUMS

In [38]:
%%time

start = time.time() 

for idx in range(66, len(ls_url)):
    print(idx)
    print(ls_url[idx])
    url = ls_url[idx]
    driver.get(url)
    time.sleep(1)
    xpath_search_ct = "//div[@class='pagination-control']/span[@class='pagination-control__description']" # this will naturally filter out NON-whiskies
    
    # scrape
    xpath_olist = "//ol[@class='spirits']/li[@class='spirit sugarcane-content']"
    whiskey_elems = driver.find_elements_by_xpath(xpath_olist)
    if len(whiskey_elems) == 0: 
        time.sleep(1)
        pass
    else:
        for i in range(len(whiskey_elems)): 
            elem = whiskey_elems[i]

            whisk_name = elem.find_element_by_xpath(".//div[@class='name']").text
            ls_whisk_name.append(whisk_name)

            whisk_type = elem.find_element_by_xpath(".//span[@class='accented-text']").text
            whisk_type = re.sub(',', '', whisk_type)
            ls_whisk_type.append(whisk_type)

            whisk_loc = elem.find_element_by_xpath(".//span[@class='location']").text
            ls_whisk_loc.append(whisk_loc)
            
            try: 
                comm_rating = elem.find_element_by_xpath(".//div[@class='community-rating']").text
                ls_comm_rating.append(comm_rating)
            except NoSuchElementException:
                comm_rating = float("NaN")
                ls_comm_rating.append(comm_rating)

            try:
                distiller_score = elem.find_element_by_xpath(".//div[@class='distiller-score']").text
            except NoSuchElementException:
                distiller_score = float("NaN")
            ls_distiller_score.append(distiller_score)

            whisk_url = elem.find_element_by_xpath("./a").get_attribute('href')
            ls_whisk_url.append(whisk_url)

            wait_time = random.randrange(5, 10)/10 ### PER REVIEW, TAKE 1s - 2s!!
            time.sleep(wait_time)



66
https://distiller.com/search?page=67&sort=total_num_of_ratings
67
https://distiller.com/search?page=68&sort=total_num_of_ratings
68
https://distiller.com/search?page=69&sort=total_num_of_ratings
69
https://distiller.com/search?page=70&sort=total_num_of_ratings
70
https://distiller.com/search?page=71&sort=total_num_of_ratings
71
https://distiller.com/search?page=72&sort=total_num_of_ratings
72
https://distiller.com/search?page=73&sort=total_num_of_ratings
73
https://distiller.com/search?page=74&sort=total_num_of_ratings
74
https://distiller.com/search?page=75&sort=total_num_of_ratings
75
https://distiller.com/search?page=76&sort=total_num_of_ratings
76
https://distiller.com/search?page=77&sort=total_num_of_ratings
77
https://distiller.com/search?page=78&sort=total_num_of_ratings
78
https://distiller.com/search?page=79&sort=total_num_of_ratings
79
https://distiller.com/search?page=80&sort=total_num_of_ratings
80
https://distiller.com/search?page=81&sort=total_num_of_ratings
81
https:/

In [39]:
driver.close()

In [ ]:
# content = driver.page_source
# whiskey_elems[0].get_attribute('innerHTML')
# print(whiskey_elems[0].text)

In [43]:
list_for_df = [ls_whisk_name, ls_whisk_type, ls_whisk_loc, ls_comm_rating, ls_distiller_score, ls_whisk_url]

df_alc_main_char = pd.DataFrame(np.array(list_for_df).T, \
    columns = ['Alc_Name', 'Alc_Type', 'Alc_Loc', 'Communal_Rating', 'Distiller_Score', 'URL'])
df_alc_main_char['Spirit'] = 'Rum'
df_alc_main_char.head()

,Alc_Name,Alc_Type,Alc_Loc,Communal_Rating,Distiller_Score,URL,Spirit
0,RON DIPLOMÁTICO RESERVA EXCLUSIVA RUM,Aged Rum,Venezuela,4.37,94,https://distiller.com/spirits/ron-diplomatico-...,Rum
1,RON ZACAPA 23 SISTEMA SOLERA RUM,Aged Rum,Guatemala,4.25,92,https://distiller.com/spirits/ron-zacapa-cente...,Rum
2,KRAKEN BLACK SPICED RUM,Spiced Rum,Trinidad & Tobago,3.59,nan,https://distiller.com/spirits/the-kraken-black...,Rum
3,PLANTATION XO 20TH ANNIVERSARY RUM,Aged Rum,Barbados,4.36,93,https://distiller.com/spirits/plantation-xo-20...,Rum
4,CAPTAIN MORGAN ORIGINAL SPICED RUM,Spiced Rum,US Virgin Islands,2.92,78,https://distiller.com/spirits/captain-morgan-o...,Rum


In [44]:
print(df_alc_main_char.shape)
df_alc_main_char = df_alc_main_char.drop_duplicates().reset_index(drop = True)
print(df_alc_main_char.shape)

(1555, 7)
(1540, 7)


In [45]:
df_alc_main_char.to_csv(os.path.dirname(os.getcwd())+'/Datasets/URLs/all_rums_main_20231210.csv', index = False)